# Random Dev For Optimizer and It's Tests

In [1]:
from numpy.testing.utils import assert_equal

from brian2 import *
import numpy as np
from brian2tools import NevergradOptimizer, SkoptOptimizer

from nevergrad.optimization import optimizerlib
from nevergrad import instrumentation as inst

WARNING    /home/alteska/anaconda3/envs/brian_opt/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)
 [py.warnings]


In [2]:
candidates, parameters = [], []

# create input and output
input_traces = np.zeros((10,5))*volt
for i in range(5):
    input_traces[5:,i]=i*10*mV

output_traces = 10*nS*input_traces

model = Equations('''
    I = g*(v-E) : amp
    g : siemens (constant)
    E : volt (constant)
    ''')

# setup the nevergrad optimizer
arg1 = inst.var.Array(1).bounded(-5, 5).asscalar()
arg2 = inst.var.Array(1).bounded(0, 10).asscalar()
instrum = inst.Instrumentation(arg1, arg2)
optim = optimizerlib.registry['DE'](instrumentation=instrum, budget=10000)

for _ in range(10):
    cand = optim.ask()
    candidates.append(cand)
    parameters.append(list(cand.args))

In [3]:
candidates[0].args

(-1.972068745163504, 3.3248429941269806)

In [4]:
x = [1,2,3]
y = [v * 5 for v in x ]
y

[5, 10, 15]

In [5]:
y = [list(v.args) for v in candidates]
y == parameters

True

In [6]:
np.random.random((10,2))

array([[0.40895349, 0.84470984],
       [0.0092116 , 0.44275525],
       [0.30910025, 0.83511014],
       [0.52191868, 0.97421928],
       [0.04958574, 0.69463674],
       [0.2863607 , 0.9504112 ],
       [0.54663335, 0.92683637],
       [0.91945383, 0.30926597],
       [0.28373822, 0.07332926],
       [0.29191423, 0.47343259]])

In [7]:
n_opt = NevergradOptimizer(bounds=[[0,1]], parameter_names=['a'])
param = n_opt.ask(10)
assert_equal(np.shape(param),(10,1))

print(np.shape(param),(10,1))


(10, 1) (10, 1)


In [8]:
param2 = param
param2

[[0.6567286901890609],
 [0.6994087184642166],
 [0.48340500498070843],
 [0.40953110623008493],
 [0.28988346532774467],
 [0.6255126163920219],
 [0.3846017762106553],
 [0.6229364388756049],
 [0.7560043935316969],
 [0.5427824795819834]]

In [9]:
np.testing.assert_array_less(param2, 1)

In [10]:
np.where(param == param2)

(array([0]),)

In [11]:
np.shape([[1,2]])

(1, 2)

In [12]:
np.shape([[1,2], [1,2]])

(2, 2)

In [13]:
error = np.random.random((10))
error

array([0.5932874 , 0.41356548, 0.29458054, 0.40931997, 0.94365416,
       0.19573792, 0.24010385, 0.35534675, 0.45390242, 0.30441813])

In [14]:
n_opt = NevergradOptimizer(bounds=[[0, 1], [0, 1]], parameter_names=['a', 'b'])
param = n_opt.ask(10)
error = np.random.random((10))

n_opt.tell(param, error)
ans = n_opt.recommend()

In [15]:
x = np.array((ans.args))
y = np.array(param)
np.where(y == x)

(array([1, 1]), array([0, 1]))

## Dev Optimizer Test

In [16]:
import numpy as np

from random import choice
from string import digits, ascii_lowercase
from numpy.testing.utils import assert_equal, assert_raises
from brian2tools import Optimizer, NevergradOptimizer, SkoptOptimizer

```
@pytest.fixture
def item():
    return Item()

def test_item_created(item):
    # ensure item was created successfully
    assert item.value == 'something expected'

def test_edit_item(item):
    # ensure item is edited successfully
    item.edit()
```

In [37]:
labels = ["".join([choice(digits + ascii_lowercase) for i in range(2)]) for j in range(10)]

bounds = np.zeros((10,2))
bounds[:, 1] = np.arange(1,11,1.)

In [39]:
for n in np.arange(1,11):
    par_names, bound = labels[:n], bounds[:n]
    n_samples = np.random.randint(1, 30)
    errors = np.random.rand(n_samples)
    print(errors)
    

    n_opt = NevergradOptimizer(bounds=bound, parameter_names=par_names)
    param = n_opt.ask(n_samples)

    assert_equal(np.shape(param), (n_samples, n))

    for i in np.arange(1, n):
        assert all(np.array(param)[:,i-1] <= i), 'Values in params are bigger than required'
        assert all(np.array(param)[:,i-1] >  0), 'Values in params are smaller than required'

[0.57067149 0.81544244 0.95585754 0.40616846]
[0.99678192 0.9948507  0.87531734 0.67799175 0.37098295 0.42773419
 0.97438486]
[0.47907073 0.40682906 0.40773303]
[0.12479966 0.3200031  0.15516998 0.56018272 0.02548262 0.91578013
 0.43695337 0.1284162  0.928594   0.66385855 0.21362237 0.9572587
 0.35778894 0.45638287 0.81439497 0.54115209 0.0393012 ]
[0.49833916 0.03033553 0.49065918 0.57914232 0.11068571 0.18414994
 0.21717572]
[0.72624199 0.85560891 0.23875312 0.529225   0.22335186]
[0.90847985 0.10922613 0.2894116  0.51170337 0.82412314 0.47946261]
[0.64929327 0.01810904 0.69565965 0.9919063  0.7715258  0.98825424]
[0.11328874 0.64610249 0.12266171 0.91278934 0.72883252 0.99632948
 0.24736336 0.77621149 0.31318939 0.08978285 0.81357236 0.20195351
 0.9683028  0.12884023 0.80022573 0.82203339 0.46417266 0.46111704
 0.05752192]
[0.64796958 0.4446012  0.48168809 0.47601102 0.69439794 0.46932521
 0.61406093 0.63151159 0.05660672 0.38741793 0.23128965 0.2770225
 0.54562425 0.32925191 0.2370

In [34]:
xx = np.zeros((10,2))
xx[:, 1] = np.arange(1,11,1.)
xx

array([[ 0.,  1.],
       [ 0.,  2.],
       [ 0.,  3.],
       [ 0.,  4.],
       [ 0.,  5.],
       [ 0.,  6.],
       [ 0.,  7.],
       [ 0.,  8.],
       [ 0.,  9.],
       [ 0., 10.]])

In [17]:
# Prep data to validate init on
chars = digits + ascii_lowercase
L = ["".join([choice(chars) for i in range(2)]) for j in range(10)]
L

a = np.arange(1, 11, 1.).reshape(10, 1)
b = np.zeros(10).reshape(10, 1)
lx = np.hstack((b, a)).tolist()
lx

In [29]:
for n in np.arange(1,11):
    par_names, bounds = L[:n], lx[:n]
    n_samples = numpy.random.randint(1, 30)
    
    n_opt = NevergradOptimizer(bounds=bounds, parameter_names=par_names)
    param = n_opt.ask(n_samples)
    
    assert_equal(np.shape(param), (n_samples,n))
    
    for i in np.arange(1, n):
        assert all(np.array(param)[:,i-1] <= i), 'Values in params are bigger than required'
        assert all(np.array(param)[:,i-1] >  0), 'Values in params are bigger than required'


In [41]:
labels = ["".join([choice(digits + ascii_lowercase) for i in range(2)]) for j in range(10)]

bounds = np.zeros((10,2))
bounds[:, 1] = np.arange(1,11,1.)

for n in np.arange(1,11):
    par_names, bound = labels[:n], bounds[:n]
    n_samples = np.random.randint(1, 30)

    n_opt = NevergradOptimizer(bounds=bound, parameter_names=par_names)
    params = n_opt.ask(n_samples)

    assert_equal(np.shape(params), (n_samples, n))

    for i in np.arange(1, n):
        assert all(np.array(params)[:,i-1] <= i), 'Values in params are bigger than required'
        assert all(np.array(params)[:,i-1] >  0), 'Values in params are smaller than required'

    errors = np.random.rand(n_samples)
    n_opt.tell(params, errors)

    print(n_opt.recommend())

Candidate(args=(0.7430915734326831,), kwargs={})
Candidate(args=(0.4239745223379151, 1.0009796418128707), kwargs={})
Candidate(args=(0.7450848772737452, 1.583312994238987, 2.4028425709906776), kwargs={})
Candidate(args=(0.21156010969457922, 0.8523259604367865, 2.208289416660266, 1.206766518289587), kwargs={})
Candidate(args=(0.5710920352418899, 0.6930313681332003, 0.5449833954230002, 2.0468247577044636, 3.323670100644775), kwargs={})
Candidate(args=(0.46706757946476635, 0.5526302376521804, 2.0753613546908434, 1.2562886107770408, 3.773310180562463, 4.098863382276798), kwargs={})
Candidate(args=(0.6176267122650366, 0.27026320481241106, 1.391639713128063, 0.940673730172108, 2.0976129626610467, 4.027804098620617, 5.656152162409917), kwargs={})
Candidate(args=(0.8656557317608836, 1.1693523256670704, 0.6857716038662696, 1.662479572841356, 3.92481621218594, 2.6477099475203896, 2.736084472113196, 2.212953038699438), kwargs={})
Candidate(args=(0.19435800570065614, 1.4603185677435973, 2.59682536

In [64]:
labels = ["".join([choice(digits + ascii_lowercase) for i in range(2)]) for j in range(10)]

bounds = np.zeros((10,2))
bounds[:, 1] = np.arange(1,11,1.)

n=1
par_names, bound = labels[:n], bounds[:n]
n_samples = np.random.randint(1, 30)

n_opt = NevergradOptimizer(bounds=bound, parameter_names=par_names)
params = n_opt.ask(n_samples)

assert_equal(np.shape(params), (n_samples, n))

for i in np.arange(1, n):
    assert all(np.array(params)[:,i-1] <= i), 'Values in params are bigger than required'
    assert all(np.array(params)[:,i-1] >  0), 'Values in params are smaller than required'

errors = np.random.rand(n_samples)
n_opt.tell(params, errors)

ans = n_opt.recommend()


In [63]:
params

[[0.1977610370905325],
 [0.2208301620340028],
 [0.5124483263022825],
 [0.39188464347413754],
 [0.6628189380258657],
 [0.6757021637383107],
 [0.5686446883939118],
 [0.207803570446564],
 [0.5698097207663052],
 [0.7408273112451442],
 [0.7607581480295477],
 [0.3058362740217061],
 [0.31457495227350163],
 [0.5736410722279344],
 [0.6428738563040859],
 [0.574936167164185],
 [0.5549457516218431],
 [0.2244571386423158],
 [0.6676922345763148],
 [0.7145285810256644],
 [0.4204387118518278],
 [0.6034307644063762],
 [0.815853978285624],
 [0.8958753591509503],
 [0.43178231578233706],
 [0.637592858484358]]

In [79]:
n_opt.optim.create_candidate.from_data(params[0])

Candidate(args=(0.585667735259693,), kwargs={}, data=[0.27582509])

In [80]:
params[0]

[0.2758250890961232]

In [72]:
n_opt.candidates[0].data

array([-0.84960767])

In [45]:
params

[[0.7075448258823988],
 [0.3385463980750758],
 [0.726343980561598],
 [0.1544080217906229],
 [0.8075598360262057],
 [0.28431613801074596],
 [0.19158976126979543],
 [0.8149153810656802],
 [0.4965109505926566],
 [0.3255620875083205],
 [0.19994066937395094],
 [0.33135382187130646],
 [0.2824447369712301],
 [0.6577895613786428],
 [0.6775996182319068],
 [0.17313441017870557],
 [0.6761165302093546],
 [0.6886980200060219],
 [0.5708827404944367],
 [0.6718468914838022],
 [0.7581727227319398],
 [0.7828959507359196],
 [0.2724542305593777],
 [0.34064116526670185]]

In [58]:
er_min = (errors).argmin()
assert params[er_min] == list(ans.args), "Optimizer didn't return the minimal value"

In [61]:
from nevergrad.optimization.base import Candidate
Candidate()

TypeError: __init__() missing 3 required positional arguments: 'args', 'kwargs', and 'data'